# Coupling BEM-BEM with Cavity.

<img src="1_Interfaz_Cav_EG.png" width="500" height="250">

To calculate the solvation energy of a solute using the BEM-BEM with cavity, which is equivalent to the BEM-BEM-BEM coupling, the following matrix system must be solved, which comes from solving the following PDE, including its border conditions:
\begin{equation} 
\left\{\begin{matrix} 
-\nabla^{2}\phi_{s_{0}}(x)+\kappa_{s}^{2}\phi_{s_{0}}(x) =0 \ & x \ \in \ \Omega_{s_{0}} \\ 
-\epsilon_{m}\nabla^{2} \phi_{m}(x)=\sum_{i=1}^{n_{c}}Q_{i}\delta\left (x-x_{i} \right ) & x \ \in \ \Omega_{m} \\
 -\nabla^{2}\phi_{s}(x)+\kappa_{s}^{2}\phi_{s}(x)=0 & x \ \in \ \Omega_{s} \\
     \phi_{s_{0}}(x)= \phi_{m}(x) & x \ \in \ \Gamma_{a} \\
     \epsilon_{s}\partial_n^a \phi_{s_{0}}(x)= \epsilon_{m}\partial_n^a \phi_{m}(x) & x \ \in \ \Gamma_{a} \\
     \phi_{m}(x)= \phi_{s}(x) & x \ \in \ \Gamma_{b} \\
     \epsilon_{m}\partial_n^b \phi_{m}(x)= \epsilon_{s}\partial_n^b \phi_{s}(x) & x \ \in \ \Gamma_{b}
\end{matrix}\right. 
\end{equation}

Where $\Omega_{m}$ is the solute domain, $\Omega_{s_0}$ is the cavity domain, $\Omega_{s}$ is the solvent domain, $\Gamma_{a}$ is the cavity-solute interface and $\Gamma_{b}$ is the solute-solvent interface. And its matrix system corresponds to:
\begin{equation}
\begin{bmatrix}
\frac{1}{2}I-K_{H,a}^{a} & V_{H,a}^{a}  & 0 & 0\\ 
\frac{1}{2}I+K_{L,a}^{a} & -\frac{\epsilon_{s}}{\epsilon_{m}}V_{L,a}^{a} &  K_{L,b}^{a} & -V_{L,b}^{a}\\ 
K_{L,a}^{b} & -\frac{\epsilon_{s}}{\epsilon_{m}}V_{L,a}^{b} &  \frac{1}{2}I+K_{L,b}^{b} & -V_{L,b}^{b}\\  
0 & 0 &  \frac{1}{2}I-K_{H,b}^{b} & \frac{\epsilon_{m}}{\epsilon_{s}}V_{H,b}^{b} 
\end{bmatrix} \cdot \begin{bmatrix}
\phi_{s_{0}}^{\Gamma_{a}} \\
\partial_n \phi_{s_{0}}^{\Gamma_{a}}\\
\phi_{m}^{\Gamma_{b}} \\
\partial_n \phi_{m}^{\Gamma_{b}}\\ 
\end{bmatrix} =
\begin{bmatrix}
0 \\
\frac{1}{4\pi\epsilon_{m}}\sum_{n=1}^{n_{c}}\frac{Q_{i}}{\left \|x_{\Gamma_{a}}-x_{i}\right \|} \\
\frac{1}{4\pi\epsilon_{m}}\sum_{n=1}^{n_{c}}\frac{Q_{i}}{\left \|x_{\Gamma_{b}}-x_{i}\right \|} \\
0 \\ 
\end{bmatrix} 
\end{equation}

And when solving the matrix system, the solvation energy is calculated as:
\begin{equation} 
    \Delta G_{Sol}=\frac{1}{2}\sum_{i=1}^{n_{c}}Q_{i} \left ( \frac{\epsilon_{s}}{\epsilon_{m}}V_L\partial_n \phi_{s_{0}}^{\Gamma_{a}}(x_{i})-K_L\phi_{s_{0}}^{\Gamma_{a}}(x_{i})+V_L\partial_n \phi_m^{\Gamma_{b}}(x_{i})-K_L\phi_m^{\Gamma_{b}}(x_{i}) \right )
\end{equation}   

## Implementation

First we import the main libraries such as Numpy, Trimesh and Bempp, as well as files in Python to read the solute files, which are in ".pqr" and ".off" format.

In [1]:
import bempp.api
import numpy as np 
import time
import trimesh
from readoff import *
from readpqr import *
start = time.time()

Next, the physical conditions of the problem in each region are defined, such as pemitivity and
Debye-Huckel parameter.

In [2]:
em = 4.    #[-] Interior electrical permittivity of the solute.
es = 80.   #[-] Exterior electrical permittivity of the solvent.
ks = 0.125  #[1/A] Inverse of the Debye-Huckel length of the fluid in the solvent and cavity.

Later, we choose the solution that we want to use to calculate the energy, where we import the surface mesh and the ".pqr" file to obtain the information on the radii, charges and position of the solute atoms.

In [3]:
Mesh1 ='Mallas_S/Sphere65R4.off' #Path of the surface mesh of the solute for which the energy is to be calculated. In "off" format
Mesh_C = 'Mallas_S/SphereCavityR4.off' #Optional: Path of the mesh with cavities, if the main mesh has the cavities, it is left blank ''.
PQR = 'PQR/Sphere5Q3.pqr' #Path of the position and charges of the solute for which the energy is to be calculated. In "pqr" format
PC,Q,R = readpqr(PQR)   

In addition, we can choose the type of preconditioner to solve the matrix system, the type of assembly of the edge operators, parameters of the GMRES and the FMM if required.

In [4]:
#Choice to solve the matrix equation in BEM/BEM with cavity.
#True: Use the Mass Matrix preconditioner.
#False: Use the Block diagonal preconditioner.
SF = False  #Strong form. 
    
#Assembly of border operators.
#fmm: For molecules with a greater number of vertices.
#default_nonlocal: For molecules with a small number of vertices.    
Assemble = 'default_nonlocal' 

#Important parameters of the GMRES.
Tol =1e-6    #GMRES tolerance.
Res =70      #Restart of the GMRES in each iteration.

#Secondary parameters when working with the fmm assembly.
bempp.api.GLOBAL_PARAMETERS.fmm.expansion_order = 5  
bempp.api.GLOBAL_PARAMETERS.fmm.ncrit = 400          
bempp.api.GLOBAL_PARAMETERS.quadrature.regular = 4   

Next, we generate the surface mesh of the solute and the cavity.

In [5]:
vertices_0,faces_0 = read_off(Mesh1) 
#In case the mesh has small gaps, with trimesh the information of the original mesh without the gaps is obtained.
meshSP = trimesh.Trimesh(vertices = vertices_0, faces= faces_0 ) 
mesh_split = meshSP.split()
print("Found %i meshes"%len(mesh_split))  #1 mesh means no cavity.

vertices_2 = mesh_split[0].vertices 
faces_2  = mesh_split[0].faces 
grid2 = bempp.api.grid.grid.Grid(vertices_2.transpose(), faces_2.transpose()) #Creation of the surface mesh of the solute-solvent interface.

if len(Mesh_C)==0:
    #Sort surface meshes of the cavities by number of vertices.
    LMS = []
    for i in range(len(mesh_split)):
        LMS.append(len(mesh_split[i].vertices))
    IMS = np.argsort(LMS)[::-1]  
    #Unite the information from the individual meshes of the cavities into a single surface mesh. 
    MS = mesh_split[IMS[1]]
    if len(mesh_split)>2: 
        for i in range(len(mesh_split)-2):
            MS = MS + mesh_split[IMS[i+2]]
    vertices_1 = MS.vertices 
    faces_1 = MS.faces  
else:    
    vertices_1,faces_1 =read_off(Mesh_C)  #Optional cavity mesh if in an '.off' file.
grid1 = bempp.api.grid.grid.Grid(vertices_1.transpose(), faces_1.transpose()) #Creation of the surface mesh with cavities.

Found 1 meshes


We make the Bempp function spaces of the potential and its derivative.

In [6]:
dirichl_space1 = bempp.api.function_space(grid1, "P", 1)  #Electrostatic potential at the interface of cavity.
neumann_space1 = bempp.api.function_space(grid1, "P", 1)  #Derived from the electrostatic potential at the interface of cavity.
dirichl_space2 = bempp.api.function_space(grid2, "P", 1)  #Electrostatic potential at the solute-solvent interface.
neumann_space2 = bempp.api.function_space(grid2, "P", 1)  #Derived from the electrostatic potential at the solute-solvent interface.

print("DS1 dofs: {0}".format(dirichl_space1.global_dof_count))
print("NS1 dofs: {0}".format(neumann_space1.global_dof_count))
print("DS2 dofs: {0}".format(dirichl_space2.global_dof_count))
print("NS2 dofs: {0}".format(neumann_space2.global_dof_count))

DS1 dofs: 180
NS1 dofs: 180
DS2 dofs: 7908
NS2 dofs: 7908


We create the boundary operators that we need for each domain.

In [7]:
#Generate the boundary operators.
#Identity operators.
I1d = bempp.api.operators.boundary.sparse.identity(dirichl_space1, dirichl_space1, dirichl_space1) # 1
I1n = bempp.api.operators.boundary.sparse.identity(dirichl_space1, neumann_space1, neumann_space1) # 1
I2d = bempp.api.operators.boundary.sparse.identity(dirichl_space2, dirichl_space2, dirichl_space2) # 1
I2n = bempp.api.operators.boundary.sparse.identity(dirichl_space2, neumann_space2, neumann_space2) # 1

#Domain of the cavity.
if ks==0:
    KHaa = bempp.api.operators.boundary.laplace.double_layer(dirichl_space1, dirichl_space1, dirichl_space1, assembler=Assemble) #K
    VHaa = bempp.api.operators.boundary.laplace.single_layer(neumann_space1, dirichl_space1, dirichl_space1, assembler=Assemble) #V
else:
    KHaa = bempp.api.operators.boundary.modified_helmholtz.double_layer(dirichl_space1, dirichl_space1, dirichl_space1, ks, assembler=Assemble) #K
    VHaa = bempp.api.operators.boundary.modified_helmholtz.single_layer(neumann_space1, dirichl_space1, dirichl_space1, ks, assembler=Assemble) #V
Z1ba = bempp.api.ZeroBoundaryOperator(dirichl_space2, dirichl_space1, dirichl_space1) #0
Z2ba = bempp.api.ZeroBoundaryOperator(neumann_space2, dirichl_space1, dirichl_space1) #0

#Domain of the solute Ωm at the interface of cavity.
KLaa = bempp.api.operators.boundary.laplace.double_layer(dirichl_space1, neumann_space1, neumann_space1, assembler=Assemble) #K
VLaa = bempp.api.operators.boundary.laplace.single_layer(neumann_space1, neumann_space1, neumann_space1, assembler=Assemble) #V
KLba = bempp.api.operators.boundary.laplace.double_layer(dirichl_space2, neumann_space1, neumann_space1, assembler=Assemble) #K
VLba = bempp.api.operators.boundary.laplace.single_layer(neumann_space2, neumann_space1, neumann_space1, assembler=Assemble) #V

#Domain of the solute Ωm at the solute-solvent interface.
KLab = bempp.api.operators.boundary.laplace.double_layer(dirichl_space1, dirichl_space2, dirichl_space2, assembler=Assemble) #K
VLab = bempp.api.operators.boundary.laplace.single_layer(neumann_space1, dirichl_space2, dirichl_space2, assembler=Assemble) #V
KLbb = bempp.api.operators.boundary.laplace.double_layer(dirichl_space2, dirichl_space2, dirichl_space2, assembler=Assemble) #K
VLbb = bempp.api.operators.boundary.laplace.single_layer(neumann_space2, dirichl_space2, dirichl_space2, assembler=Assemble) #V

#Domain of the solvent Ωs.
Z1ab = bempp.api.ZeroBoundaryOperator(dirichl_space1, neumann_space2, neumann_space2) #0
Z2ab = bempp.api.ZeroBoundaryOperator(neumann_space1, neumann_space2, neumann_space2) #0
if ks==0:
    KHbb = bempp.api.operators.boundary.laplace.double_layer(dirichl_space2, neumann_space2, neumann_space2, assembler=Assemble) #K
    VHbb = bempp.api.operators.boundary.laplace.single_layer(neumann_space2, neumann_space2, neumann_space2, assembler=Assemble) #V
else:
    KHbb = bempp.api.operators.boundary.modified_helmholtz.double_layer(dirichl_space2, neumann_space2, neumann_space2, ks, assembler=Assemble) #K
    VHbb = bempp.api.operators.boundary.modified_helmholtz.single_layer(neumann_space2, neumann_space2, neumann_space2, ks, assembler=Assemble) #V

Later, we create the Coulomb potential function $\phi_{c}^{\Gamma}$ and with that, we construct the vector right head side $b$ of formulation.
\begin{equation}
\phi_{c}^{\Gamma} =\frac{1}{4\pi\epsilon_{m}}\sum_{n=1}^{n_{c}}\frac{Q_{i}}{\left \|x_{\Gamma}-x_{i}\right \|} \quad \quad
b=\begin{bmatrix}
0 \\
\phi_{c}^{\Gamma_{a}} \\
\phi_{c}^{\Gamma_{b}} \\
0 \\
\end{bmatrix}
\end{equation}

In [8]:
@bempp.api.complex_callable(jit=False)
def U_c(x, n, domain_index, result):
    global Q,PC,em
    result[:] = (1 / (4.*np.pi*em))  * np.sum( Q / np.linalg.norm( x - PC, axis=1))
Uc1 = bempp.api.GridFunction(neumann_space1, fun=U_c)  
Uc2 = bempp.api.GridFunction(dirichl_space2, fun=U_c)  

#Construction of the right vector.
if SF==False:
    # Rhs in cavity.
    rhs_S0 = np.zeros(dirichl_space1.global_dof_count) #0   
    # Rhs in Ωm at the interface of cavity.
    rhs_M1 = (Uc1).projections(neumann_space1) # uc    
    # Rhs in Ωm at the solute-solvent interface.
    rhs_M2 = (Uc2).projections(dirichl_space2) # uc    
    # Rhs in Ωs
    rhs_S = np.zeros(neumann_space2.global_dof_count) #0
    # The combination of Rhs.
    rhs = np.concatenate([rhs_S0, rhs_M1, rhs_M2, rhs_S])
else:
    Uc0 = bempp.api.GridFunction(dirichl_space1, fun=U_c) 
    rhs = [0*I1d*Uc0, I1n*Uc0, I2d*Uc2, 0*I2n*Uc2] 

And we construct the global left 4x4 matrix $A$ of the formulation.   
\begin{equation}
A=\begin{bmatrix}
\frac{1}{2}I-K_{H,a}^{a} & V_{H,a}^{a}  & 0 & 0\\ 
\frac{1}{2}I+K_{L,a}^{a} & -\frac{\epsilon_{s}}{\epsilon_{m}}V_{L,a}^{a} &  K_{L,b}^{a} & -V_{L,b}^{a}\\ 
K_{L,a}^{b} & -\frac{\epsilon_{s}}{\epsilon_{m}}V_{L,a}^{b} &  \frac{1}{2}I+K_{L,b}^{b} & -V_{L,b}^{b}\\  
0 & 0 &  \frac{1}{2}I-K_{H,b}^{b} & \frac{\epsilon_{m}}{\epsilon_{s}}V_{H,b}^{b} 
\end{bmatrix}
\end{equation}

In [9]:
if SF==False:
    #Position of the 4x4 matrix.
    blocks = [[None,None,None,None],[None,None,None,None],[None,None,None,None],[None,None,None,None]] 
    blocks[0][0] = (0.5*I1d-KHaa).weak_form()  
    blocks[0][1] = VHaa.weak_form()            
    blocks[0][2] = Z1ba.weak_form()            
    blocks[0][3] = Z2ba.weak_form()       
    
    blocks[1][0] = (0.5*I1n+KLaa).weak_form()  
    blocks[1][1] = -(es/em)*VLaa.weak_form()   
    blocks[1][2] = KLba.weak_form()            
    blocks[1][3] = -VLba.weak_form()   
    
    blocks[2][0] = KLab.weak_form()           
    blocks[2][1] = -(es/em)*VLab.weak_form()   
    blocks[2][2] = (0.5*I2d+KLbb).weak_form()  
    blocks[2][3] = -VLbb.weak_form()   
    
    blocks[3][0] = Z1ab.weak_form()           
    blocks[3][1] = Z2ab.weak_form()            
    blocks[3][2] = (0.5*I2n-KHbb).weak_form()  
    blocks[3][3] = (em/es)*VHbb.weak_form()    
    blocked = bempp.api.assembly.blocked_operator.BlockedDiscreteOperator(np.array(blocks))     
    #Block diagonal preconditioner for BEM.
    from preconditioners import *
    P = BlockDiagonal_4x4_Cavidad(dirichl_space1, neumann_space1, dirichl_space2, neumann_space2, blocks, es,em,ks)
else:
    #Position of the 4x4 matrix.
    blocks = bempp.api.BlockedOperator(4,4)  
    blocks[0,0] = (0.5*I1d-KHaa) 
    blocks[0,1] = VHaa           
    blocks[0,2] = Z1ba           
    blocks[0,3] = Z2ba         
    
    blocks[1,0] = (0.5*I1n+KLaa) 
    blocks[1,1] = -(es/em)*VLaa  
    blocks[1,2] = KLba           
    blocks[1,3] = -VLba   
    
    blocks[2,0] = KLab           
    blocks[2,1] = -(es/em)*VLab 
    blocks[2,2] = (0.5*I2d+KLbb) 
    blocks[2,3] = -VLbb         
    
    blocks[3,0] = Z1ab           
    blocks[3,1] = Z2ab           
    blocks[3,2] = (0.5*I2n-KHbb) 
    blocks[3,3] = (em/es)*VHbb   

Later, we solve the matrix system $Ax=b$ with GMRES together with its respective preconditioning an efficient solution. The solution is then divided into the parts associated with $\phi_{s_{0}}^{\Gamma_{a}}$, $\partial_{n}\phi_{s_{0}}^{\Gamma_{a}}$, $\phi_{m}^{\Gamma_{b}}$ and $\partial_{n}\phi_{m}^{\Gamma_{b}}$.

In [10]:
#Iteration counter.
it_count = 0
def count_iterations(x):
    global it_count
    it_count += 1
    if (it_count / 10) == (it_count // 10):
        print(it_count,x)
          
# Solution by GMRES.
from scipy.sparse.linalg import gmres
if SF==False:
    start1 = time.time()
    soln, info = gmres(blocked, rhs, M=P, callback=count_iterations,tol=Tol, restart=Res)  
    end1 = time.time() 
else:
    start1 = time.time()
    soln, info, res, it_count = bempp.api.linalg.gmres(blocks, rhs, return_residuals=True, return_iteration_count=True, use_strong_form=True,tol=Tol, restart=Res)
    end1 = time.time() 
    
# Time to solve the equation.
curr_time1 = (end1 - start1)
print("Number of GMRES iterations: {0}".format(it_count))
print("Total time in GMRES: {:5.2f} [s]".format(curr_time1))

10 0.024783643837071753
20 0.001657854496711443
30 0.0002556873822083932
40 7.743710414812609e-05
50 2.7123286618677927e-05
60 7.0509102609102005e-06
Number of GMRES iterations: 66
Total time in GMRES: 43.92 [s]


Next, we make Bempp functions from the solution of $\phi_{s_{0}}^{\Gamma_{a}}$, $\partial_{n}\phi_{s_{0}}^{\Gamma_{a}}$, $\phi_{m}^{\Gamma_{b}}$ and $\partial_{n}\phi_{m}^{\Gamma_{b}}$.

In [11]:
if SF==False:
    soln_u1 = soln[:dirichl_space1.global_dof_count]
    soln_du1 = soln[dirichl_space1.global_dof_count : dirichl_space1.global_dof_count + neumann_space1.global_dof_count]
    soln_u2 =  soln[dirichl_space1.global_dof_count + neumann_space1.global_dof_count : dirichl_space1.global_dof_count + neumann_space1.global_dof_count + dirichl_space2.global_dof_count]
    soln_du2 = soln[dirichl_space1.global_dof_count + neumann_space1.global_dof_count + dirichl_space2.global_dof_count:]
    # Solution to the function with Dirichlet in the cavity interface.
    dirichlet_fun1 = bempp.api.GridFunction(dirichl_space1, coefficients=soln_u1)
    # Solution of the function with Neumann at the cavity interface.
    neumann_fun1 = bempp.api.GridFunction(neumann_space1, coefficients=soln_du1)
    # Solution of the function with Dirichlet at the solute-solvent interface.
    dirichlet_fun2 = bempp.api.GridFunction(dirichl_space2, coefficients=soln_u2)
    # Solution of the function with Neumann at the solute-solvent interface.
    neumann_fun2 = bempp.api.GridFunction(neumann_space2, coefficients=soln_du2)
else:
    dirichlet_fun1 = soln[0] 
    neumann_fun1 = soln[1] 
    dirichlet_fun2 = soln[2] 
    neumann_fun2 = soln[3]  

Finally we calculate the solvation energy for the BEM-BEM coupling with cavity, using the equation of the potential at the position of the solute atoms.
\begin{equation}
\phi_m(x) =\frac{\epsilon_{s}}{\epsilon_{m}}V_L\partial_{n}\phi_{s_{0}}^{\Gamma_{a}}(x)-K_L\phi_{s_{0}}^{\Gamma_{a}}(x)+V_L\partial_{n}\phi_m^{\Gamma_{b}}(x)-K_L\phi_m^{\Gamma_{b}}(x)  \quad \quad
 \Delta G_{Sol}=\frac{1}{2}\sum_{i=1}^{n_{c}}Q_{i}\phi_{m}(x_i) 
\end{equation} 

In [12]:
#Result of the total solvation energy.
VF1 = bempp.api.operators.potential.laplace.single_layer(neumann_space1, np.transpose(PC)) 
KF1 = bempp.api.operators.potential.laplace.double_layer(dirichl_space1, np.transpose(PC))
VF2 = bempp.api.operators.potential.laplace.single_layer(neumann_space2, np.transpose(PC)) 
KF2 = bempp.api.operators.potential.laplace.double_layer(dirichl_space2, np.transpose(PC))
uF = VF1*neumann_fun1*(es/em) - KF1*dirichlet_fun1 + VF2*neumann_fun2 - KF2*dirichlet_fun2 
E_Solv = 0.5*4.*np.pi*332.064*np.sum(Q*uF).real 
print('Solvation Energy: {:7.6f} [kCal/mol]'.format(E_Solv) )

#Total time.
end = time.time()
curr_time = (end - start)
print("Total time: {:5.2f} [s]".format(curr_time))

Solvation Energy: -92.484818 [kCal/mol]
Total time: 228.71 [s]
